In [177]:
import pandas as pd
import numpy
import math

#IMPORT GENERATORS DATA FROM NREL, INCLUDES PLANT LEVEL HEAT REATES(HR). https://data.nrel.gov/submissions/100

#plant level data with 4th-level polynomial heat rate curve
hr_curves_201617 = pd.read_csv("/Users/Michael/Downloads/CEMS_2016-2017_Heat_Rates.csv")

#plants in Texas
TX_hr_curves_201617 = hr_curves_201617.loc[hr_curves_201617["state"] == "TX"]
#fossil plant ids for Texas from NREL data
NREL_TX_fossil_ids = TX_hr_curves_201617['unit_id']
NREL_TX_fossil_ids = [float(x.split('_', 1)[0]) for x in NREL_TX_fossil_ids]


#plants in ERCOT
ERCOT_hr_curves_201617 = hr_curves_201617.loc[hr_curves_201617["NERC_region"] == "ERCOT"]
NREL_ERCOT_fossil_ids = ERCOT_hr_curves_201617['unit_id']

#WHY IS THERE SUCH A BIG DIFFERENCE BETWEEN THE NUMBER IN EACH?

#plant level data with piece-wise linear heat rate function. DOES NOT INDICATE STATE OR NERC REGION. 
#Next step: cross reference IDs to select just TX/ERCOT plants
hr_pwlinear_201516 = pd.read_csv("/Users/Michael/Downloads/CEMS_2015-2016_Piecewise_Fits.csv")

In [178]:
#Plant ids from Generators_Data.csv

#Full Generators_data.csv from OneZone_TX
GenX_TX_gens = pd.read_csv("/Users/Michael/GenX-Projects/Example_Systems/OneZone_TX/Generators_data.csv")

#Plant ids from Generates_data (includes fossil fuels and renewables)
GenX_TX_gens_IDs = pd.DataFrame(GenX_TX_gens["plant_id_eia"])

GenX_TX_fossil_gens = GenX_TX_gens.loc[(GenX_TX_gens["technology"] == "Conventional Steam Coal") | (GenX_TX_gens["technology"] == "Natural Gas Fired Combined Cycle") | (GenX_TX_gens["technology"] == "Natural Gas Fired Combustion Turbine") | (GenX_TX_gens["technology"] == "Natural Gas Steam Turbine")]

#Create list of fossil generator ids
GenX_TX_fossil_ids = []
for x in GenX_TX_fossil_gens['plant_id_eia']:
    GenX_TX_fossil_ids = GenX_TX_fossil_ids + x[1:-1].split(",")

GenX_TX_fossil_ids = [float(x) for x in GenX_TX_fossil_ids]

In [179]:
#ERCOT hourly load data, https://www.ercot.com/gridinfo/load/load_hist

#Files from ERCOT, include sub-zone load and total load
Load_ERCOT_2016 = pd.read_excel("/Users/Michael/Downloads/ERCOT_Load_2016.xlsx")['ERCOT']
Load_ERCOT_2017 = pd.read_excel("/Users/Michael/Downloads/ERCOT_Load_2016.xlsx")['ERCOT']
Load_ERCOT_2018 = pd.read_excel("/Users/Michael/Downloads/ERCOT_Load_2016.xlsx")['ERCOT']
Load_ERCOT_2019 = pd.read_excel("/Users/Michael/Downloads/ERCOT_Load_2016.xlsx")['ERCOT']

In [180]:
#ERCOT hourly renewables generation
from pandas import DataFrame

#Annual excel files with renewable generaiton data, 15-minute intervals
#FORMAT CHANGES AFTER 2016!!!!
ERCOT_gen_excel_2016 = pd.ExcelFile("/Users/Michael/Downloads/FuelMixReport_PreviousYears/IntGenByFuel2016.xlsx")
ERCOT_gen_excel_2017 = pd.ExcelFile("/Users/Michael/Downloads/FuelMixReport_PreviousYears/IntGenByFuel2017.xlsx")
ERCOT_gen_excel_2018 = pd.ExcelFile("/Users/Michael/Downloads/FuelMixReport_PreviousYears/IntGenByFuel2018.xlsx")
ERCOT_gen_excel_2019 = pd.ExcelFile("/Users/Michael/Downloads/FuelMixReport_PreviousYears/IntGenByFuel2019.xlsx")

#Past-2016 FORMATTING

def format_gen_data_until_2016 (gen_data: pd.ExcelFile):
    #create list of sheet names containing renewable generation data
    gen_data.sheet_names
    sheets = list(gen_data.sheet_names[2:])
    sheets.reverse()

    #make a dataframe with all the sheets combined
    months_ren_gen = []

    for sheet in sheets:
        month = pd.DataFrame(gen_data.parse(sheet))
        months_ren_gen.append(month)

    ERCOT_ren_gen = pd.concat(months_ren_gen, ignore_index = True)

    #change column names to string
    col_names = ERCOT_ren_gen.columns
    ERCOT_ren_gen.columns = [str(x) for x in col_names]

    #replace nan values in 00:00:00 with 1900-01-01 00:00:00 values
    for i in range(len(ERCOT_ren_gen["Date-Fuel"])):
        if math.isnan(ERCOT_ren_gen['00:00:00'][i]) == True:
            ERCOT_ren_gen['00:00:00'][i] = ERCOT_ren_gen['1900-01-01 00:00:00'][i]
    
    return ERCOT_ren_gen

In [173]:
#2017-PRESENT FORMATTING

def format_gen_data_post_2016 (gen_data: pd.ExcelFile):
    #create list of sheet names containing renewable generation data
    gen_data.sheet_names
    sheets = list(gen_data.sheet_names[4:])

    months_ren_gen = []

    for sheet in sheets:
        month = pd.DataFrame(gen_data.parse(sheet))
        months_ren_gen.append(month)

    ERCOT_ren_gen = pd.concat(months_ren_gen, ignore_index = True)

    #change column names to string
    col_names = ERCOT_ren_gen.columns
    ERCOT_ren_gen.columns = [str(x) for x in col_names]

    return(ERCOT_ren_gen)


In [193]:
#function that returns hourly generation for wind and solar for data from 2016 and before
def combine_gen_data_pre_2016(gen_data: DataFrame):

    wind_gen = []
    solar_gen = []

    #for i in range(24):
    for i in range(len(gen_data["Date-Fuel"])):
        if gen_data["Date-Fuel"][i].split("_")[-1] == "Solar":
            for hour in range(int(24)):
                hourly_solar = 0
                for period in range(4):
                    hourly_solar = hourly_solar + gen_data.iloc[i][2 + 4*hour + period]
                solar_gen.append(hourly_solar)
        
        if gen_data["Date-Fuel"][i].split("_")[-1] == "Wind":
            for hour in range(int(24)):
                hourly_wind = 0
                for period in range(4):
                    hourly_wind = hourly_wind + gen_data.iloc[i][2 + 4*hour + period]
                wind_gen.append(hourly_wind)

    return solar_gen, wind_gen

#function that returns hourly generation for wind and solar for data from after 2016
def combine_gen_data_post_2016(gen_data: DataFrame):

    wind_gen = []
    solar_gen = []

    #for i in range(24):
    for i in range(len(gen_data["Fuel"])):
        if gen_data["Fuel"][i] == "Solar":
            for hour in range(int(24)):
                hourly_solar = 0
                for period in range(4):
                    hourly_solar = hourly_solar + gen_data.iloc[i][5 + 4*hour + period]
                solar_gen.append(hourly_solar)
        
        if gen_data["Fuel"][i] == "Wind":
            for hour in range(int(24)):
                hourly_wind = 0
                for period in range(4):
                    hourly_wind = hourly_wind + gen_data.iloc[i][5 + 4*hour + period]
                wind_gen.append(hourly_wind)

    return solar_gen, wind_gen

In [192]:
ERCOT_ren_gen.iloc[0][5]

12.245844

In [197]:
#Create the modified Generators_variability_modified file based on the Generators_Variability givne in Texas One Zone

gen_var_mod = pd.read_csv("/Users/Michael/GenX-Projects/Example_Systems/OneZone_TX/Generators_variability_modified.csv")
gen_data_mod = pd.read_csv("/Users/Michael/GenX-Projects/Example_Systems/OneZone_TX/Generators_data_modified.csv")

#from https://www.ercot.com/gridinfo/resource/2020, released Jan 2020
wind_cap_2019 = 23860
solar_cap_2019 = 2281

#use functinos above to generate renewable production timeeseries
#ERCOT_ren_gen = format_gen_data_post_2016(ERCOT_gen_excel_2019)
solar_gen, wind_gen = combine_gen_data_post_2016(ERCOT_ren_gen)

#weight of renewable production (hourly production/total capacity)
weighted_solar_gen_2019 = [x / solar_cap_2019 for x in solar_gen]
weighted_wind_gen_2019 = [x / wind_cap_2019 for x in wind_gen]

max(weighted_solar_gen_2019)
max(weighted_wind_gen_2019)


0.8192294403604358

In [199]:
print(
max(weighted_solar_gen_2019),
max(weighted_wind_gen_2019))

#SNL energy to get gas and coal prices, ask for a license
#compare the dispatch to the actualy ERCOt dispatch, sanity check by fuel type
#down the road, let's check out the impact of nuclear off-lining
#Hydro-quebec has some kind of variability booster


0.8653184769837792 0.8192294403604358
